In [5]:

from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import re



In [6]:
def cleantxt(txt):
    
    stpw = ['about', 'above', 'across', 'after', 'afterwards', 'again', 'against', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'among', 'amongst', 'amoungst', 'amount', 'and', 'another', 'any', 'anyhow', 'anyone', 'anything', 'anyway', 'anywhere', 'are', 'around', 'back', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'behind', 'being', 'below', 'beside', 'besides', 'between', 'beyond', 'bill', 'both', 'bottom', 'but', 'call', 'can', 'cannot', 'cant', 'con', 'could', 'couldnt', 'cry', 'describe', 'detail', 'done', 'down', 'due', 'during', 'each', 'eight', 'either', 'eleven', 'else', 'elsewhere', 'empty', 'enough', 'etc', 'even', 'ever', 'every', 'everyone', 'everything', 'everywhere', 'except', 'few', 'fifteen', 'fify', 'fill', 'find', 'fire', 'first', 'five', 'for', 'former', 'formerly', 'forty', 'found', 'four', 'from', 'front', 'full', 'further', 'get', 'give', 'had', 'has', 'hasnt', 'have', 'hence', 'her', 'here', 'hereafter', 'hereby', 'herein', 'hereupon', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'however', 'hundred', 'inc', 'indeed', 'interest', 'into', 'its', 'itself', 'keep', 'last', 'latter', 'latterly', 'least', 'less', 'ltd', 'made', 'many', 'may', 'meanwhile', 'might', 'mill', 'mine', 'more', 'moreover', 'most', 'mostly', 'move', 'much', 'must', 'myself', 'name', 'namely', 'neither', 'never', 'nevertheless', 'next', 'nine', 'nobody', 'none', 'noone', 'nor', 'not', 'nothing', 'now', 'nowhere', 'off', 'often', 'once', 'one', 'only', 'onto', 'other', 'others', 'otherwise', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 'part', 'per', 'perhaps', 'please', 'put', 'rather', 'same', 'see', 'seem', 'seemed', 'seeming', 'seems', 'serious', 'several', 'she', 'should', 'show', 'side', 'since', 'sincere', 'six', 'sixty', 'some', 'somehow', 'someone', 'something', 'sometime', 'sometimes', 'somewhere', 'still', 'such', 'system', 'take', 'ten', 'than', 'that', 'the', 'their', 'them', 'themselves', 'then', 'thence', 'there', 'thereafter', 'thereby', 'therefore', 'therein', 'thereupon', 'these', 'they', 'thickv', 'thin', 'third', 'this', 'those', 'though', 'three', 'through', 'throughout', 'thru', 'thus', 'together', 'too', 'top', 'toward', 'towards', 'twelve', 'twenty', 'two', 'under', 'until', 'upon', 'very', 'via', 'was', 'well', 'were', 'what', 'whatever', 'when', 'whence', 'whenever', 'where', 'whereafter', 'whereas', 'whereby', 'wherein', 'whereupon', 'wherever', 'whether', 'which', 'while', 'whither', 'who', 'whoever', 'whole', 'whom', 'whose', 'why', 'will', 'with', 'within', 'without', 'would', 'yet', 'you', 'your', 'yours', 'yourself', 'yourselves', 'the', 'com']
    txt = re.sub(r"\n", " ", txt)
    txt = re.sub("[\<\[].*?[\>\]]", " ", txt)
    txt = txt.lower()
    txt = re.sub(r"[^a-z ]", " ", txt)
    txt = re.sub(r"\b\w{1,2}\b", " ",txt)
    txt = " ".join([x for x in txt.split() if x not in stpw])
    return txt



In [20]:
def load_data():

    df = pd.read_csv("data.txt")
    df['comment_text'] = df.comment_text.apply(lambda x : cleantxt(x))
    
    X = df.iloc[:,1]
    y = df.iloc[:,2:]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=123)

    X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=123)

    return X_train, X_val, X_test, y_train, y_val, y_test


In [21]:
X_train, X_val, X_test, y_train, y_val, y_test = load_data()

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(decode_error='ignore',stop_words='english')
train_tfidf = vect.fit_transform(X_train)
val_tfidf = vect.transform(X_val)
test_tfidf = vect.transform(X_test)

In [23]:
col = ['toxic', 'severe_toxic', 'obscene', 'threat','insult', 'identity_hate']
pred_train = np.zeros((X_train.shape[0],len(col)))
pred_test = np.zeros((X_test.shape[0],len(col)))
pred_val = np.zeros((X_val.shape[0],len(col)))

In [24]:
from sklearn.linear_model import LogisticRegression
LogR = LogisticRegression()

In [25]:
for i,x in enumerate(col):
    LogR.fit(train_tfidf, y_train[x])
    pred_train[:,i] = LogR.predict_proba(train_tfidf)[:,1]
    pred_val[:,i] = LogR.predict_proba(val_tfidf)[:,1]
    pred_test[:,i] = LogR.predict_proba(test_tfidf)[:,1]
    print(x,"predicted!")

C:\Users\Dev\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


toxic predicted!
severe_toxic predicted!
obscene predicted!
threat predicted!
insult predicted!
identity_hate predicted!


In [26]:
AUC = np.zeros((3,6))
AUC

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [27]:
from sklearn import metrics
for i,x in enumerate(col):
    auc = np.array([metrics.roc_auc_score(y_train[x], pred_train[:,i]),
                    metrics.roc_auc_score(y_val[x], pred_val[:,i]),
                    metrics.roc_auc_score(y_test[x], pred_test[:,i])])
    print(x,"Train AUC:",auc[0],", Val AUC:",auc[1],", Test AUC:",auc[2])
    AUC[:,i] = auc

avg_auc = AUC.mean(axis=1)
print("Average Train AUC:",avg_auc[0],", Average Val AUC:",avg_auc[1],", Average Test AUC:",avg_auc[2])

toxic Train AUC: 0.9856740292915671 , Val AUC: 0.969288204777488 , Test AUC: 0.9676377412562506
severe_toxic Train AUC: 0.992689722913682 , Val AUC: 0.9857203795160635 , Test AUC: 0.981633391904646
obscene Train AUC: 0.9939668440125141 , Val AUC: 0.9822841003158724 , Test AUC: 0.987497090155988
threat Train AUC: 0.9956640450995974 , Val AUC: 0.9716062449414582 , Test AUC: 0.9817858432405686
insult Train AUC: 0.9886224678955288 , Val AUC: 0.9739582964447687 , Test AUC: 0.9757563308948265
identity_hate Train AUC: 0.9915038958373484 , Val AUC: 0.9759493696932049 , Test AUC: 0.9650255800032955
Average Train AUC: 0.9913535008417061 , Average Val AUC: 0.9764677659481427 , Average Test AUC: 0.9765559962425959
